In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

## Read data

Load eCLIP-seq sequences

In [3]:
eclip = pd.read_csv("data/HNRNPC_eclip.txt", sep='\t', header=None)

In [4]:
eclip.head()

,0,1
0,chr7:138151219-138151300,TTTTTTTGGAGACAGCATCTCACTGTGTTGCCCATACTGGAGTGCA...
1,chr7:139051735-139051798,ATTTTTTGGAGACAGAGTCTCTGTCTCCCAGGCGTGAAATGCAGTG...
2,chr7:918222-918272,TTTTTTGCGACGGAGTCTCGCTCTGTCCTCTGTCCCAGGCTGGAGT...
3,chr7:138154088-138154142,TTTTTGAGACAGTGTCTCGTGCAGTGGCCATCTTGGCTCACTGCAA...
4,chr7:138167839-138167908,TTTTTGTAGCGGGGGGGGTCTCACTTTGTTGCCCAGGCTGGTGTCA...


Load RBNS binding affinities

In [5]:
four_mers = pd.read_csv("data/HNRNPC_4.tsv", sep='\t')
five_mers = pd.read_csv("data/HNRNPC_5.tsv", sep='\t',)
six_mers = pd.read_csv("data/HNRNPC_6.tsv", sep='\t',)
four_mers

,[hnRNPC],0,5,20,80,320,1300
0,TTTT,1.050108,1.213564,1.234794,2.755430,11.477239,5.958550
1,ATTT,1.052096,1.124278,0.990108,1.255270,2.505993,1.978384
2,CTTT,1.094396,1.111101,1.087047,1.272162,2.264619,1.867528
3,TTTA,1.023649,1.074103,0.975594,1.183402,2.129294,1.799562
4,TTTC,1.097361,1.150883,1.052075,1.217492,1.999443,1.715131
...,...,...,...,...,...,...,...
251,TCGG,0.830022,0.846412,0.842917,0.818757,0.693297,0.672110
252,TCGT,0.849285,0.844630,0.870984,0.837076,0.685304,0.681841
253,CGTC,0.845628,0.842135,0.878028,0.833263,0.678240,0.691214
254,ATCG,0.849546,0.843249,0.866847,0.828345,0.666320,0.674459


Load knockdown differential rna-seq data

In [6]:
diff_exp = pd.read_csv("data/HNRNPC.tsv", sep='\t',)

In [7]:
diff_exp.head()

,test_id,gene_id,gene,locus,sample_1,sample_2,status,value_1,value_2,log2(fold_change),test_stat,p_value,q_value,significant
0,XLOC_000001,XLOC_000001,DDX11L1,chr1:11868-31109,HNRNPC-BGHLV20-HepG2,Control,NOTEST,0.000000,0.021689,inf,0.0,1.0,1.0,no
1,XLOC_000002,XLOC_000002,MIR1302-11,chr1:11868-31109,HNRNPC-BGHLV20-HepG2,Control,NOTEST,0.017697,0.000000,-inf,0.0,1.0,1.0,no
2,XLOC_000003,XLOC_000003,OR4G4P,chr1:52472-54936,HNRNPC-BGHLV20-HepG2,Control,NOTEST,0.000000,0.000000,0.0,0.0,1.0,1.0,no
3,XLOC_000004,XLOC_000004,OR4G11P,chr1:62947-63887,HNRNPC-BGHLV20-HepG2,Control,NOTEST,0.000000,0.000000,0.0,0.0,1.0,1.0,no
4,XLOC_000005,XLOC_000005,OR4F5,chr1:69090-70008,HNRNPC-BGHLV20-HepG2,Control,NOTEST,0.000000,0.000000,0.0,0.0,1.0,1.0,no


In [8]:
diff_exp.shape

(48097, 14)

## Generate positive pairs

EClip positive binding sites

In [9]:
in_vivo_pos_seqs = eclip[1].values

RBNS significant binding affinity

In [10]:
def get_signficant_sequences(df):
    num_cols = df.columns[1:]
    filtered = df[df[num_cols].gt(2).any(axis=1)]
    pairs = [
        (row[df.columns[0]], tuple(row[num_cols]))
        for _, row in filtered.iterrows()
    ]
    return pairs

In [11]:
pos_four_mers = get_signficant_sequences(four_mers)
pos_five_mers = get_signficant_sequences(five_mers)
pos_six_mers = get_signficant_sequences(six_mers)

In [12]:
pos_four_mers

[('TTTT', (1.050108, 1.213564, 1.234794, 2.75543, 11.477239, 5.95855)),
 ('ATTT', (1.052096, 1.124278, 0.990108, 1.25527, 2.505993, 1.978384)),
 ('CTTT', (1.094396, 1.111101, 1.087047, 1.272162, 2.264619, 1.867528)),
 ('TTTA', (1.023649, 1.074103, 0.975594, 1.183402, 2.129294, 1.799562))]

In [13]:
in_vitro_pos_seqs = np.concatenate((pos_four_mers,pos_five_mers,pos_six_mers))
in_vitro_pos_seqs.shape

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


(117, 2)

Filter diff expr

In [14]:
# remove when gene expression is always 0 or if logfold change is inf/-inf
diff_exp_filtered = diff_exp[
    ~(
        ((diff_exp["value_1"] == 0) & (diff_exp["value_2"] == 0)) |
        (diff_exp["log2(fold_change)"].isin([np.inf, -np.inf]))
    )
]

In [15]:
diff_exp_filtered.head()

,test_id,gene_id,gene,locus,sample_1,sample_2,status,value_1,value_2,log2(fold_change),test_stat,p_value,q_value,significant
7,XLOC_000008,XLOC_000008,OR4F29,chr1:317719-461954,HNRNPC-BGHLV20-HepG2,Control,OK,0.42339,1.53871,1.861660,0.889161,0.00805,0.035235,yes
11,XLOC_000012,XLOC_000012,MTND1P23,chr1:536815-660283,HNRNPC-BGHLV20-HepG2,Control,OK,410.13200,336.58000,-0.285138,-0.149730,0.81500,0.897404,no
12,XLOC_000013,XLOC_000013,MTND2P28,chr1:536815-660283,HNRNPC-BGHLV20-HepG2,Control,OK,931.02300,834.29300,-0.158262,-0.292230,0.66870,0.802839,no
13,XLOC_000014,XLOC_000014,hsa-mir-6723,chr1:536815-660283,HNRNPC-BGHLV20-HepG2,Control,OK,1147.94000,1761.41000,0.617688,1.819070,0.00955,0.040072,yes
14,XLOC_000015,XLOC_000015,RP5-857K21.7,chr1:536815-660283,HNRNPC-BGHLV20-HepG2,Control,OK,1200.71000,1284.82000,0.097686,0.164563,0.81710,0.898417,no


In [16]:
diff_exp_filtered.shape

(20355, 14)

In [17]:
logfold_changes = diff_exp_filtered['log2(fold_change)'].values
logfold_changes

array([ 1.86166 , -0.285138, -0.158262, ..., -1.65698 ,  0.128098,
       -1.54899 ])

Build positive inputs and outputs

In [18]:
import itertools
import random
combinations_size = 100000
pos_input_sequence_combinations = [
    (random.choice(in_vivo_pos_seqs), random.choice(in_vitro_pos_seqs))
    for _ in range(combinations_size)
]

In [19]:
pos_labels = [("HNRNPC", logfold_changes)] * len(pos_input_sequence_combinations)

In [20]:
pos_input_sequence_combinations

[('GTCTCTCAGACCCCTGGATTCAGAACCCAAGGCCA',
  array(['CAATTT',
         (1.070547, 1.135088, 0.985793, 1.256936, 2.55934, 2.090553)],
        dtype=object)),
 ('TTTAAAGAGACACAGTCTTGCTCTGACACTCAGACT',
  array(['TTTATT',
         (1.083366, 1.142882, 1.114286, 1.765261, 5.720871, 3.707175)],
        dtype=object)),
 ('AACAGGATCTTGCTGTGTTGCCTAGGCTGGTCTGAACTCGGGCCTAAG',
  array(['ACTTT',
         (1.079541, 1.106275, 1.055918, 1.26789, 2.318257, 1.913072)],
        dtype=object)),
 ('GATCTTGCTCTGTTGCCCAGGCTGGAGTGCCGCTGGTGCAATCA',
  array(['TTTCC',
         (1.266323, 1.326392, 1.191603, 1.374251, 2.180355, 1.961175)],
        dtype=object)),
 ('GAGAAGTCAGAAAGTTAGTAAGGGGTGTGTGCCATATAC',
  array(['TTTTG',
         (0.998612, 1.108458, 0.988141, 1.381256, 3.599706, 2.629749)],
        dtype=object)),
 ('GGGGAAGAACTCTAAGTCCCTCCTACACTACTAGCTTCCATGGAAAAGCTAGTAGAAAACAA',
  array(['TTTTGA',
         (1.083472, 1.191468, 1.051302, 1.402626, 3.312505, 2.622345)],
        dtype=object)),
 ('CAAACATAGTTG

## Generate negative pairs

Shuffle eCLIP sequences

In [21]:
import random
def shuffle_by_pairs(seq):
    pairs = [seq[i:i+2] for i in range(0, len(seq), 2)]
    random.shuffle(pairs)
    return "".join(pairs)
in_vivo_neg_seqs = [shuffle_by_pairs(seq) for seq in eclip[1].values]

Retrieve RBNS sequences that are not enriched

In [22]:
def get_insignficant_sequences(df):
    num_cols = df.columns[1:]
    filtered = df[df[num_cols].lt(2).any(axis=1)]
    pairs = [
        (row[df.columns[0]], tuple(row[num_cols]))
        for _, row in filtered.iterrows()
    ]
    return pairs

In [23]:
neg_four_mers = get_insignficant_sequences(four_mers)
neg_five_mers = get_insignficant_sequences(five_mers)
neg_six_mers = get_insignficant_sequences(six_mers)

In [24]:
neg_four_mers

[('TTTT', (1.050108, 1.213564, 1.234794, 2.75543, 11.477239, 5.95855)),
 ('ATTT', (1.052096, 1.124278, 0.990108, 1.25527, 2.505993, 1.978384)),
 ('CTTT', (1.094396, 1.111101, 1.087047, 1.272162, 2.264619, 1.867528)),
 ('TTTA', (1.023649, 1.074103, 0.975594, 1.183402, 2.129294, 1.799562)),
 ('TTTC', (1.097361, 1.150883, 1.052075, 1.217492, 1.999443, 1.715131)),
 ('TATT', (1.039198, 1.06593, 1.000795, 1.150161, 1.713124, 1.496541)),
 ('TTTG', (1.004893, 1.050304, 0.987782, 1.093798, 1.596789, 1.390267)),
 ('GTTT', (0.973983, 1.046386, 0.956951, 1.079698, 1.564893, 1.351425)),
 ('CCCC', (1.419824, 1.45442, 1.343596, 1.41312, 1.549043, 1.601944)),
 ('TTAT', (1.011911, 1.034474, 0.975124, 1.09913, 1.524726, 1.373946)),
 ('ACCC', (1.48522, 1.480864, 1.389412, 1.432278, 1.504673, 1.613837)),
 ('TTCT', (1.115392, 1.128724, 1.087663, 1.167728, 1.499969, 1.362751)),
 ('CACC', (1.425643, 1.40017, 1.396304, 1.407528, 1.478731, 1.612886)),
 ('TCTT', (1.067138, 1.066816, 1.070433, 1.137513, 1.47739,

In [25]:
in_vitro_neg_seqs = np.concatenate((neg_four_mers,neg_five_mers,neg_six_mers))
in_vitro_neg_seqs.shape

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


(5375, 2)

Construct placeholder expression change vector

In [26]:
placeholder_logfold_changes = np.zeros(shape=logfold_changes.shape)

Build negative inputs and outputs

In [27]:
neg_input_sequence_combinations = [
    (random.choice(in_vivo_neg_seqs), random.choice(in_vitro_neg_seqs))
    for _ in range(combinations_size)
]

In [28]:
neg_labels = [("Not HNRNPC", placeholder_logfold_changes)] * len(neg_input_sequence_combinations)

## Split into train/val/test

In [29]:
# concatenate data
inputs = np.concatenate((pos_input_sequence_combinations, neg_input_sequence_combinations))


In [30]:
outputs = np.concatenate((np.ones(len(pos_input_sequence_combinations)), np.zeros(len(neg_input_sequence_combinations))))

In [31]:
inputs.shape

(200000, 2)

In [32]:
from sklearn.model_selection import GroupShuffleSplit


# eCLIP sequences
groups = [item[0] for item in inputs]  
# RBP labels 0/1
y = np.array(outputs).astype('int')

# train split
gss1 = GroupShuffleSplit(n_splits=1, train_size=0.7, random_state=42)
train_idx, temp_idx = next(gss1.split(X=np.zeros(len(y)), y=y, groups=groups))

# val+test
y_temp = y[temp_idx]
groups_temp = np.array(groups)[temp_idx]

# val + test split
gss2 = GroupShuffleSplit(n_splits=1, train_size=0.5, random_state=42)
val_idx_rel, test_idx_rel = next(gss2.split(X=np.zeros(len(y_temp)), y=y_temp, groups=groups_temp))

val_idx = temp_idx[val_idx_rel]
test_idx = temp_idx[test_idx_rel]

In [33]:
print("Train labels distribution:", np.bincount(y[train_idx]))
print("Val labels distribution:", np.bincount(y[val_idx]))
print("Test labels distribution:", np.bincount(y[test_idx]))

# Make sure sequences don't overlap
assert len(set(np.array(groups)[train_idx]) & set(np.array(groups)[val_idx])) == 0
assert len(set(np.array(groups)[train_idx]) & set(np.array(groups)[test_idx])) == 0
assert len(set(np.array(groups)[val_idx]) & set(np.array(groups)[test_idx])) == 0

Train labels distribution: [70050 69975]
Val labels distribution: [15189 14873]
Test labels distribution: [14761 15152]


In [34]:
train_data = [inputs[i] for i in train_idx]
val_data   = [inputs[i] for i in val_idx]
test_data  = [inputs[i] for i in test_idx]
train_labels = y[train_idx]
val_labels = y[val_idx]
test_labels = y[test_idx]

In [35]:
# get positive/negative label values
positive_value = ("HNRNPC", logfold_changes)
negative_value = ("Not HNRNPC", placeholder_logfold_changes)

## Save as dictionaries

Save as pickle files since there are nested data structures

In [36]:
import pickle

In [37]:
'''
Saved data format for each split:
- Input
    - List containing:
        - eCLIP sequence: text
        - Tuple of RBNS sequence and binding affinity: (text, vector)
- Output
    - List containing:
        - RBP label: text
        - Logfold gene expression change: vector
        
Labels save separately to save storage:
- List containing:
    - RBP label: text
    - Logfold gene expression change: vector
'''

'\nSaved data format for each split:\n- Input\n    - List containing:\n        - eCLIP sequence: text\n        - Tuple of RBNS sequence and binding affinity: (text, vector)\n- Output\n    - List containing:\n        - RBP label: text\n        - Logfold gene expression change: vector\n        \nLabels save separately to save storage:\n- List containing:\n    - RBP label: text\n    - Logfold gene expression change: vector\n'

In [38]:
# Save train
with open("data/train_split.pkl", "wb") as f:
    pickle.dump((train_data, train_labels), f)

In [39]:
# Save val
with open("data/val_split.pkl", "wb") as f:
    pickle.dump((val_data, val_labels), f)

In [40]:
# Save test
with open("data/test_split.pkl", "wb") as f:
    pickle.dump((test_data, test_labels), f)

In [41]:
# Save positive label
with open("data/HNRNPC_positive_label.pkl", "wb") as f:
    pickle.dump(positive_value, f)

In [42]:
# Save negative label
with open("data/HNRNPC_negative_label.pkl", "wb") as f:
    pickle.dump(negative_value, f)